In [1]:
import pandas as pd 
import numpy as np
import collections
pd.set_option('display.max_columns', None)

In [3]:
# first make bike accident dataset

fileName = "data/source data/accidents.csv"
usecolsall = ["X","Y","OBJECTID","CRIMEID","CCN","REPORTDATE","ROUTEID","MEASURE","OFFSET","STREETSEGID","ROADWAYSEGID","FROMDATE","TODATE","ADDRESS","LATITUDE","LONGITUDE","XCOORD","YCOORD","WARD","EVENTID","MAR_ADDRESS","MAR_SCORE","MAJORINJURIES_BICYCLIST","MINORINJURIES_BICYCLIST","UNKNOWNINJURIES_BICYCLIST","FATAL_BICYCLIST","MAJORINJURIES_DRIVER","MINORINJURIES_DRIVER","UNKNOWNINJURIES_DRIVER","FATAL_DRIVER","MAJORINJURIES_PEDESTRIAN","MINORINJURIES_PEDESTRIAN","UNKNOWNINJURIES_PEDESTRIAN","FATAL_PEDESTRIAN","TOTAL_VEHICLES","TOTAL_BICYCLES","TOTAL_PEDESTRIANS","PEDESTRIANSIMPAIRED","BICYCLISTSIMPAIRED","DRIVERSIMPAIRED","TOTAL_TAXIS","TOTAL_GOVERNMENT","SPEEDING_INVOLVED","NEARESTINTROUTEID","NEARESTINTSTREETNAME","OFFINTERSECTION","INTAPPROACHDIRECTION","LOCATIONERROR","LASTUPDATEDATE","MPDLATITUDE","MPDLONGITUDE","MPDGEOX","MPDGEOY"]
accidentData = pd.read_csv(fileName, usecols=usecolsall)
accidentData['FROMDATE'] = pd.to_datetime(accidentData['FROMDATE']) 
accidentData = accidentData[accidentData['FROMDATE'] > "2016-12-31"]
accidentData['FROMDATE'] = pd.to_datetime(accidentData['FROMDATE']) 
accidentData = accidentData[accidentData['FROMDATE'] > "2016-12-31"]

smallData = accidentData[['TOTAL_BICYCLES','TOTAL_VEHICLES','TOTAL_PEDESTRIANS','LATITUDE', 'LONGITUDE', 'XCOORD', 'YCOORD', 'FATAL_BICYCLIST', 'MPDLATITUDE', 'MPDLONGITUDE', 'FROMDATE','STREETSEGID','ROUTEID','NEARESTINTSTREETNAME','NEARESTINTROUTEID','OFFINTERSECTION']]

# get only accidents that include a bike
bikeData = smallData[smallData['TOTAL_BICYCLES'] > 0]

# save to csv
bikeData.to_csv("data/modified data/bikes.csv", index=False)

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# now make bike share movement data
bikeShareTrips = pd.read_csv("data/source data/bikeShareTrips.csv")
bikeShareLocations = pd.read_csv("data/source data/bikeShareLocations.csv")
startLocations = bikeShareTrips['start_station_name'].values
endLocations = bikeShareTrips['end_station_name'].values
locationList = bikeShareLocations['NAME'].apply(lambda x:x.lower().strip())

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
b = 0
for i in locationList:
    b = b + 1
    
fr = []
for ss, es in zip(bikeShareTrips['start_station_name'], bikeShareTrips['end_station_name']):
    fr.append(str(ss) + ' - ' + str(es))
bikeShareTrips['fullRoutes'] = fr

start_station_names = []
end_station_names = []
total_rides = []
start_station_lats = []
start_station_longs = []
end_station_lats = []
end_station_longs = []

for i,j in zip(bikeShareTrips['fullRoutes'].value_counts().index,bikeShareTrips['fullRoutes'].value_counts()):
    
    start_station_name = (i.split("-")[0]).strip()
    start_station_names.append(start_station_name)
    end_station_name = (i.split("-")[1]).strip()
    end_station_names.append(end_station_name)
    
    start_station_row = bikeShareLocations[bikeShareLocations['NAME'] == start_station_name]
    end_station_row = bikeShareLocations[bikeShareLocations['NAME'] == end_station_name]

    if (len(start_station_row['LATITUDE'].values) > 0):
        start_station_lats.append(start_station_row['LATITUDE'].values[0])
    else:
        start_station_lats.append(None)
        
    if (len(start_station_row['LONGITUDE'].values) > 0):
        start_station_longs.append(start_station_row['LONGITUDE'].values[0])
    else:
        start_station_longs.append(None)
        
    if (len(end_station_row['LATITUDE'].values) > 0):
        end_station_lats.append(end_station_row['LATITUDE'].values[0])
    else:
        end_station_lats.append(None)
        
    if (len(end_station_row['LONGITUDE'].values) > 0):
        end_station_longs.append(end_station_row['LONGITUDE'].values[0])
    else:
        end_station_longs.append(None)
    
    total_rides.append(j)
    
bikeShareMovement = pd.DataFrame(
            {
                "start_station_name" : start_station_names,
                "end_station_name" : end_station_names,
                "total_ride" : total_rides,
                "start_station_lat" : start_station_lats,
                "start_station_long" : start_station_longs,
                "end_station_lat" : end_station_lats,
                "end_station_long" : end_station_longs
            })

bikeShareMovement = bikeShareMovement[bikeShareMovement['start_station_lat'] != None]
bikeShareMovement = bikeShareMovement[bikeShareMovement['start_station_long'] != None]
bikeShareMovement = bikeShareMovement[bikeShareMovement['end_station_lat'] != None]
bikeShareMovement = bikeShareMovement[bikeShareMovement['end_station_long'] != None]
bikeShareMovement = bikeShareMovement[bikeShareMovement['start_station_lat'].notna()]
bikeShareMovement = bikeShareMovement[bikeShareMovement['start_station_long'].notna()]
bikeShareMovement = bikeShareMovement[bikeShareMovement['end_station_lat'].notna()]
bikeShareMovement = bikeShareMovement[bikeShareMovement['end_station_long'].notna()]

bikeShareMovement.head()
bikeShareMovement.to_csv("data/modified data/bikeShareMovement.csv")

In [6]:
# now make bike total traffic data
bikeShareTrips = pd.read_csv("data/source data/bikeShareTrips.csv")
bikeShareLocations = pd.read_csv("data/source data/bikeShareLocations.csv")

locationInRides = []
locationOutRides = []
locationNames = []
locationIds = []
locationLats = []
locationLongs = []
locationBikes = []

startCounter = collections.Counter(bikeShareTrips['start_station_name'])
endCounter = collections.Counter(bikeShareTrips['end_station_name'])

for location_id, location_name, location_lat, location_long, location_capacity in zip(bikeShareLocations["STATION_ID"],bikeShareLocations["NAME"], bikeShareLocations["LATITUDE"],bikeShareLocations["LONGITUDE"],bikeShareLocations["CAPACITY"]):
    startCount = 0
    endCount = 0
    startCount = startCounter.get(location_name)
    endCount = endCounter.get(location_name)
            
    locationInRides.append(startCount)
    locationOutRides.append(endCount)
    locationNames.append(location_name)
    locationIds.append(location_id)
    locationLats.append(location_lat)
    locationLongs.append(location_long)
    locationBikes.append(location_capacity)

bikeShareStationTotals = pd.DataFrame(
            {
                "station" : locationNames,
                "station_id" : locationIds,
                "total_out" : locationOutRides,
                "total_in" : locationInRides,
                "station_lat" : locationLats,
                "station_long" : locationLongs,
                "station_bikes" : locationBikes
            })

bikeShareStationTotals.to_csv("data/modified data/bikeShareStationTotals.csv")

In [7]:
bikeShareStationTotals

,station,station_id,total_out,total_in,station_lat,station_long,station_bikes
0,5th & K St NW,31,18616.0,18496.0,38.903040,-77.019027,19
1,19th & East Capitol St SE,32,3204.0,3186.0,38.889600,-76.976900,15
2,Park Rd & Holmead Pl NW,33,6511.0,7836.0,38.930800,-77.031500,35
3,Good Hope & Naylor Rd SE,34,96.0,189.0,38.861663,-76.969184,19
4,Connecticut Ave & Macomb St NW / Cleveland Park,35,NaN,NaN,38.933626,-77.057530,11
...,...,...,...,...,...,...,...
672,N. Beauregard St. & Berkley St.,629,481.0,473.0,38.840797,-77.108938,19
673,Mt Vernon Trail & S. Washington St.,630,2494.0,2485.0,38.793065,-77.049436,15
674,Eisenhower Ave & Ike Dr,632,355.0,386.0,38.802738,-77.082463,15
675,Capitol Ave & Kendall St NE,633,1217.0,1261.0,38.910993,-76.985854,19
